# MeanReverter Strategy Batch Optimization

这个notebook用于对多个交易对进行MeanReverter策略的批量优化。

In [1]:
# 导入必要的库...
import os
import pandas as pd
from datetime import datetime
from glob import glob
import backtrader as bt 
import optuna
import warnings
import quantstats as qs
warnings.filterwarnings('ignore')
from IPython.display import clear_output
from pathlib import Path  # 使用pathlib代替os
# 添加必要的导入
from concurrent.futures import ThreadPoolExecutor
# _data_loading_executor = ThreadPoolExecutor(max_workers=10)  # 用于数据加载的线程池

# 遍历每个交易对，但使用多进程方式
from joblib import Parallel, delayed
import numpy as np

# 在此处添加全局缓存字典，用于缓存数据加载结果和数据完整性检查结果
_data_feed_cache = {}
_data_completeness_cache = {}

c:\Users\x7498\anaconda3\envs\backtrader\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from MeanReverterShort import MeanReverterShort

# 在 CONFIG 中添加所有需要动态配置的参数
CONFIG = {
    # 策略相关配置
    'strategy': {
        'class': MeanReverterShort,
        'name': MeanReverterShort.__name__
    },
    
    # 数据相关配置（单币种、单时间周期）
    # 如果 selected_symbols 为空，则通过 get_all_symbols 自动获取所有交易对
    'selected_symbols': [
    "BTCUSDT",
    "ETHUSDT",
    "SOLUSDT",
    "XRPUSDT",
    "DOGEUSDT",
    "BNBUSDT",
    "OMUSDT",
    "ADAUSDT",
    "ENAUSDT",
    "LTCUSDT",
    "SUIUSDT",
    "1000PEPEUSDT",
    "AVAXUSDT",
    "LINKUSDT",
    "AAVEUSDT",
    "TRXUSDT",
    "ONDOUSDT",
    "WLDUSDT",
    "DOTUSDT",
    "APTUSDT",
    "UNIUSDT",
    "FILUSDT",
    "JUPUSDT",
    "1000SHIBUSDT",
    "SEIUSDT",
    "ARBUSDT",
    "ATOMUSDT",
    "LDOUSDT",
    "ALCHUSDT",
    "OPUSDT",],
    
    'data_path': r'\\znas\Main\futures',
    'start_date': '2024-01-01',
    'end_date': '2025-02-08',
    'source_timeframe': '1m',
    # 针对批量优化使用多个目标时间周期
    'target_timeframes': ['15min'],
    
    # 文件保存配置
    'reports_path': 'reports',
    'results_filename_template': 'optimization_results_{strategy_name}_{start_date}-{end_date}.csv',
    
    # 回测参数配置
    'commission': 0.0004,
    'initial_capital': 10000,
    # 如果需要可以添加：
    # 'trade_on_close': True,
    # 'exclusive_orders': True,
    # 'hedging': False,
    
    # 优化参数配置，根据 MeanReverterShort 策略的参数进行优化
    'optimization_params': {
        'frequency': range(8, 16, 2),            # 用于计算慢速 RSI 均线的周期（默认10），步长为2
        'rsiFrequency': range(38, 46, 2),         # 计算 RSI 的周期（默认40），步长为2
        'sellZoneDistance': range(3, 8, 1),       # 卖出区域距离（默认5），步长为1
        'avgUpATRSum': range(3, 8, 1),            # ATR求和周期（默认3），步长为1
        'useAbsoluteRSIBarrier': [True, False],   # 是否使用绝对 RSI 阈值进行平仓（默认True）
        'barrierLevel': range(44, 57, 2),          # 障碍水平（默认50），步长为2
        'pyramiding': range(6, 11, 1)              # 最大加仓次数（默认8），步长为1
    },
    
    # 优化设置
    'optimization_settings': {
        'n_trials': 160,       # 可根据需要调整试验次数
        'min_trades': 50,
        'timeout': 3600,
        'n_jobs': 30           # -1 表示使用所有 CPU 核心; 也可以设置为具体的数量
    },

}

In [3]:
def get_timeframe_params(timeframe_str):
    """
    将时间周期字符串转换为 backtrader 的 timeframe 和 compression 参数
    """
    if timeframe_str.endswith('min'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('min', '')))
    elif timeframe_str.endswith('H'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('H', '')) * 60)
    elif timeframe_str.endswith('D'):
        return (bt.TimeFrame.Days, 1)
    elif timeframe_str == '1m':
        return (bt.TimeFrame.Minutes, 1)
    else:
        raise ValueError(f"不支持的时间周期格式: {timeframe_str}")



def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=r'\\znas\Main\futures'):
    """
    加载并重采样期货数据，并缓存已经重采样后的 DataFrame 以避免重复 I/O 操作
    """
    # 构造缓存键
    key = (symbol, start_date, end_date, source_timeframe, target_timeframe, data_path)
    if key in _data_feed_cache:
        # 如果缓存中有，返回新的数据馈送对象（注意拷贝，防止被修改）
        cached_df = _data_feed_cache[key]
        timeframe, compression = get_timeframe_params(target_timeframe)
        data_feed = bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date),
            todate=pd.to_datetime(end_date)
        )
        
        # 添加clone方法，这样可以快速创建数据副本而不需要重新执行IO
        data_feed.clone = lambda: bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date),
            todate=pd.to_datetime(end_date)
        )
        
        return data_feed
    
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 顺序读取文件，不使用线程池
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并、排序，以及重采样数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    combined_df.set_index('datetime', inplace=True)
    
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    backtesting_df = backtesting_df.dropna()
    
    # 将结果缓存在全局变量中（使用拷贝，以免后续被修改）
    _data_feed_cache[key] = backtesting_df.copy()
    
    timeframe, compression = get_timeframe_params(target_timeframe)
    data_feed = bt.feeds.PandasData(
        dataname=backtesting_df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    # 添加clone方法
    data_feed.clone = lambda: bt.feeds.PandasData(
        dataname=backtesting_df.copy(),
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    return data_feed

In [4]:
def get_all_symbols(data_path, date_str):
    """获取指定日期目录下的所有交易对"""
    daily_path = os.path.join(data_path, date_str)
    if not os.path.exists(daily_path):
        return []
    
    files = glob(os.path.join(daily_path, f"{date_str}_*_USDT_1m.csv"))
    symbols = set()  # 使用 set 进行去重
    for file in files:
        filename = os.path.basename(file)
        symbol = filename.split('_')[1]
        symbols.add(symbol)
    return list(symbols)

def verify_data_completeness(symbol, start_date, end_date, data_path):
    """验证数据完整性"""
    # 构造缓存键
    key = (symbol, start_date, end_date, data_path)
    if key in _data_completeness_cache:
        return _data_completeness_cache[key]
    
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        file_path = os.path.join(
            data_path,
            date_str,
            f"{date_str}_{formatted_symbol}_USDT_1m.csv"  # 文件名格式保持不变
        )
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}")
            _data_completeness_cache[key] = False
            return False
    _data_completeness_cache[key] = True
    return True

In [5]:
# 添加自定义评分函数
def custom_score(strat):
    """
    自定义评分函数 - 以最大化回报率为主要目标
    保留最低交易次数要求作为基本约束
    """
    # 获取交易次数
    trades = strat.analyzers.trades.get_analysis()
    total_trades = trades.get('total', {}).get('total', 0)
    
    # 从returns分析器获取总回报率
    returns = strat.analyzers.returns.get_analysis()
    total_return = returns.get('rtot', 0) * 100  # 转为百分比
    
    # 交易次数惩罚 - 确保策略至少有足够的交易
    min_trades = CONFIG['optimization_settings'].get('min_trades', 50)
    trade_penalty = 1.0 if total_trades >= min_trades else (total_trades / min_trades) ** 2
    
    # 最终得分简单地使用调整后的回报率
    # 交易次数不足的策略会受到严厉惩罚
    score = total_return * trade_penalty
    
    return score

In [6]:
def load_master_results(config):
    """
    加载全局优化结果文件，并提取已完成优化的组合（基于 'Symbol', 'Target Timeframe', 'Rank' 列）。
    """
    # 构造 master 文件路径
    start_clean = config['start_date'].replace("-", "")
    end_clean = config['end_date'].replace("-", "")
    master_file = os.path.join(
        config['reports_path'], 
        config['results_filename_template'].format(
            strategy_name=config['strategy']['name'],
            start_date=start_clean,
            end_date=end_clean
        )
    )
    
    if os.path.exists(master_file):
        try:
            master_df = pd.read_csv(master_file)
        except Exception as e:
            try:
                master_df = pd.read_excel(master_file)
            except Exception as e2:
                print(f"无法读取文件: {e}, {e2}")
                master_df = pd.DataFrame()
    else:
        master_df = pd.DataFrame()
    
    optimized_combinations = set()
    if not master_df.empty:
        if {'Target Timeframe', 'Symbol', 'Rank'}.issubset(master_df.columns):
            for symbol in master_df['Symbol'].unique():
                for tf in master_df['Target Timeframe'].unique():
                    rows = master_df[(master_df['Symbol'] == symbol) & (master_df['Target Timeframe'] == tf)]
                    ranks = rows['Rank'].tolist()
                    # 当存在 5 个排名且排名为 1 到 5 时认为该组合已完成优化
                    if len(ranks) == 5 and set(ranks) == set(range(1, 6)):
                        optimized_combinations.add((symbol, tf))
        else:
            print("警告: 结果文件缺少必要的列，将重新开始优化")
    else:
        print("优化结果文件为空")
        
    return master_file, master_df, optimized_combinations

def save_master_results(new_results, master_file):
    """
    将新的优化结果追加到全局结果文件中，只进行追加操作，并避免重复数据。
    """
    if not new_results:
        print("警告: 没有新的结果需要保存")
        return

    new_df = pd.DataFrame(new_results)
    
    try:
        # 如果文件不存在或为空，直接写入
        if not os.path.exists(master_file) or os.path.getsize(master_file) == 0:
            # 确保目录存在
            os.makedirs(os.path.dirname(master_file), exist_ok=True)
            new_df.to_csv(master_file, index=False)
            print(f"创建新文件并写入 {len(new_df)} 条记录")
            return
        
        # 读取现有数据
        try:
            existing_df = pd.read_csv(master_file)
            if existing_df.empty:
                # 如果文件存在但为空，直接写入新数据
                new_df.to_csv(master_file, index=False)
                print(f"文件为空，写入 {len(new_df)} 条新记录")
                return
        except pd.errors.EmptyDataError:
            # 如果文件存在但为空，直接写入新数据
            new_df.to_csv(master_file, index=False)
            print(f"文件为空，写入 {len(new_df)} 条新记录")
            return
        
        # 创建用于检查重复的键
        def create_key(row):
            return f"{row['Symbol']}_{row['Target Timeframe']}_{row['Rank']}"
        
        existing_keys = set(existing_df.apply(create_key, axis=1))
        new_df['_temp_key'] = new_df.apply(create_key, axis=1)
        
        # 过滤掉重复的记录
        truly_new_df = new_df[~new_df['_temp_key'].isin(existing_keys)]
        truly_new_df = truly_new_df.drop('_temp_key', axis=1)
        
        if len(truly_new_df) > 0:
            # 追加非重复的新数据
            truly_new_df.to_csv(master_file, mode='a', header=False, index=False)
            print(f"成功追加 {len(truly_new_df)} 条新记录（过滤掉 {len(new_df) - len(truly_new_df)} 条重复记录）")
        else:
            print("所有记录都已存在，无需追加")
            
    except Exception as e:
        print(f"处理数据时出错: {str(e)}")
        # 如果出错，尝试直接写入新文件
        try:
            backup_file = f"{master_file}.backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            if os.path.exists(master_file):
                os.rename(master_file, backup_file)
                print(f"已将问题文件备份为: {backup_file}")
            new_df.to_csv(master_file, index=False)
            print(f"已创建新文件并写入 {len(new_df)} 条记录")
        except Exception as e2:
            print(f"备份和重写文件时出错: {str(e2)}")
        return

    # 验证最终结果
    try:
        full_df = pd.read_csv(master_file)
        print(f"当前文件总共包含 {len(full_df)} 条记录")
    except Exception as e:
        print(f"验证追加结果时出错: {str(e)}")

In [7]:
def optimize_strategy(symbol, timeframe, master_file):
    """
    使用 Optuna 优化策略参数，直接保存最优的前 5 个参数组合
    """
    if not master_file:
        raise ValueError("master_file 参数不能为空")
    
    # ... 其余代码保持不变 ...
    # ... 其他代码保持不变 ...
    # 1. 预加载数据，只执行一次IO操作
    preloaded_data = load_and_resample_data(
        symbol, CONFIG['start_date'], CONFIG['end_date'],
        target_timeframe=timeframe
    )
    
    study = optuna.create_study(
        study_name=f"{symbol}_{timeframe}",
        direction="maximize",
        storage=None  # 使用内存存储
    )
    
    print(f"开始优化 {symbol}-{timeframe}...")
    
    def objective(trial):
        try:
            params = {}
            for param_name, param_range in CONFIG['optimization_params'].items():
                if isinstance(param_range, range):
                    params[param_name] = trial.suggest_int(
                        param_name,
                        param_range.start,
                        param_range.stop - 1,
                        param_range.step
                    )
                else:
                    params[param_name] = trial.suggest_categorical(param_name, param_range)
            
            cerebro = bt.Cerebro(
                        optdatas=True,    
                        optreturn=True,   
                        runonce=True,     
                        preload=True      
            )
            data = preloaded_data.clone()
            cerebro.adddata(data)
            cerebro.addstrategy(CONFIG['strategy']['class'], **params)
            
            cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
            cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
            
            results = cerebro.run()
            strat = results[0]
            score = custom_score(strat)
            
            print(f"[{symbol}-{timeframe}] Trial {trial.number}: 参数 {params} -> 得分 {score:.2f}")
            
            return score
        except Exception as e:
            print(f"[{symbol}-{timeframe}] Trial {trial.number} 出错: {e}")
            return float('-inf')
    
    study.optimize(
        objective,
        n_trials=CONFIG['optimization_settings']['n_trials'],
        timeout=CONFIG['optimization_settings']['timeout'],
        n_jobs=CONFIG['optimization_settings'].get('n_jobs', 1),
        catch=(Exception,)
    )
    
    # 提取前 5 个最佳试验并直接保存
    completed_trials = [
        t for t in study.trials
        if t.state == optuna.trial.TrialState.COMPLETE and t.value is not None and t.value > float('-inf')
    ]
    
    if not completed_trials:
        print(f"警告: {symbol}-{timeframe} 没有有效的完成试验")
        return []
    
    top_trials = sorted(completed_trials, key=lambda t: t.value, reverse=True)[:5]
    
    # 准备保存结果
    results_to_save = []
    for rank, trial in enumerate(top_trials, 1):
        result = {
            'Symbol': symbol,
            'Target Timeframe': timeframe,
            'Rank': rank,
            'Score': trial.value,
            **trial.params  # 展开参数
        }
        results_to_save.append(result)
    
    # 直接保存结果
    save_master_results(results_to_save, master_file)
    
    print(f"[{symbol}-{timeframe}] 已保存最佳参数组合")
    return results_to_save

In [8]:


# def process_symbol_tf(symbol, tf, config):
#     """
#     针对单个交易对和指定时间周期执行策略参数优化与回测，并赋予 1~5 的排名。
#     """
#     print(f"\n开始针对 {symbol} 时间周期 {tf} 优化...")
#     # 使用已有的 optimize_strategy 函数
#     top_results = optimize_strategy(symbol, tf)
#     if not top_results:
#         print(f"警告: {symbol} 在 {tf} 时间周期下优化失败")
#         return []
    
#     processed_results = []
#     # 为每个参数组合运行回测并获取量化指标，同时赋予排名
#     for idx, res in enumerate(top_results, start=1):
#         res['Symbol'] = symbol
#         res['Target Timeframe'] = tf
#         res['Rank'] = idx
#         metrics = run_backtest_with_params(res, symbol, tf)
#         res.update(metrics)
#         processed_results.append(res)
        
#     print(f"完成 {symbol} 在 {tf} 时间周期下的优化，获得 {len(processed_results)} 个结果")
#     return processed_results



def auto_backup_results(master_file, backup_folder='backups'):
    """
    创建优化结果文件的自动备份
    """
    if not os.path.exists(master_file):
        print(f"文件不存在，无需备份: {master_file}")
        return
    
    # 创建备份目录
    os.makedirs(backup_folder, exist_ok=True)
    
    # 创建带时间戳的备份文件名
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.basename(master_file)
    backup_file = os.path.join(backup_folder, f"{filename}.{timestamp}.bak")
    
    try:
        import shutil
        shutil.copy2(master_file, backup_file)
        print(f"已自动创建备份: {backup_file}")
    except Exception as e:
        print(f"自动备份失败: {str(e)}")

def batch_optimize(config):
    """
    批量优化所有交易对和指定多个时间周期
    """

    # 使用多线程并行处理
    import threading
    from queue import Queue

    os.makedirs(config['reports_path'], exist_ok=True)
    
    master_file, master_df, optimized_combinations = load_master_results(config)
    
    # 获取交易对列表
    if config.get('selected_symbols'):
        symbols = config['selected_symbols']
    else:
        symbols = get_all_symbols(config['data_path'], config['start_date'])
    
    # 创建备份
    auto_backup_results(master_file)
    
    # 准备未完成的组合
    remaining_combinations = []
    for symbol in symbols:
        for tf in config['target_timeframes']:
            if (symbol, tf) not in optimized_combinations:
                remaining_combinations.append((symbol, tf))
    
    print(f"剩余需要优化的组合数量: {len(remaining_combinations)} 个")
    
    # 使用多线程处理
    task_queue = Queue()
    for combo in remaining_combinations:
        task_queue.put(combo)
    
    file_lock = threading.Lock()
    
    def worker():
        while not task_queue.empty():
            try:
                symbol, tf = task_queue.get(block=False)
                print(f"开始处理组合: {symbol}-{tf}")
                with file_lock:
                    optimize_strategy(symbol, tf, master_file)  # 传入 master_file
                task_queue.task_done()
            except Exception as e:
                print(f"处理任务出错: {e}")
                try:
                    task_queue.task_done()
                except:
                    pass
    
    # 创建并启动线程
    n_threads = min(30, len(remaining_combinations))
    threads = []
    for _ in range(n_threads):
        t = threading.Thread(target=worker)
        t.daemon = True
        threads.append(t)
        t.start()
    
    # 等待所有线程完成
    for t in threads:
        t.join()
    
    # 最终备份
    auto_backup_results(master_file, backup_folder='backups_final')
    print("批量优化完成")


In [9]:
# def process_symbol_batch(symbol, timeframes, config_path):
#     """用于并行处理的包装函数，避免传递大型配置对象"""
#     try:
#         # 在子进程中重新加载配置，避免序列化问题
#         import json
#         with open(config_path, 'r') as f:
#             config = json.load(f)
            
#         # 导入必要的模块（每个进程需要单独导入）
#         import os
#         import pandas as pd
#         import optuna
#         import backtrader as bt
        
#         results = []
        
#         print(f"开始处理交易对: {symbol}")
#         # 验证数据完整性
#         if not verify_data_completeness(symbol, config['start_date'], config['end_date'], config['data_path']):
#             print(f"警告: 跳过 {symbol} - 数据不完整")
#             return []
        
#         # 处理每个时间周期
#         for tf in timeframes:
#             print(f"处理组合: {symbol}-{tf}")
#             # 使用内存存储优化
#             top_results = optimize_strategy(symbol, tf)
#             if not top_results:
#                 continue
                
#             processed_results = []
#             # 为每个参数组合运行回测并获取量化指标
#             for idx, res in enumerate(top_results, start=1):
#                 res['Symbol'] = symbol
#                 res['Target Timeframe'] = tf
#                 res['Rank'] = idx
#                 metrics = run_backtest_with_params(res, symbol, tf)
#                 res.update(metrics)
#                 processed_results.append(res)
            
#             results.extend(processed_results)
#             print(f"完成组合 {symbol}-{tf} 的优化")
        
#         return results
#     except Exception as e:
#         import traceback
#         error_msg = f"处理 {symbol} 时出错: {str(e)}\n{traceback.format_exc()}"
#         print(error_msg)
#         # 返回错误信息而不是引发异常
#         return [{"error": error_msg, "Symbol": symbol}]

In [10]:
# def run_backtest_with_params(params, symbol, timeframe):
#     """
#     使用指定参数运行策略的回测并计算收益指标（利用 quantstats）。
#     返回一个包含基础回测指标和所有 quantstats 指标的字典。
#     """
#     # 过滤掉不属于策略参数部分的键（如 'score', 'symbol', 'timeframe'等）
#     valid_keys = set(CONFIG["optimization_params"].keys())
#     strategy_params = {k: v for k, v in params.items() if k in valid_keys}

#     cerebro = bt.Cerebro(
#                 optdatas=True,    # 启用数据优化
#                 optreturn=True,   # 仅返回必要结果
#                 runonce=True,     # 批处理模式
#                 preload=True      # 预加载数据
#     )
#     data = load_and_resample_data(symbol, CONFIG['start_date'], CONFIG['end_date'],
#                                   target_timeframe=timeframe)
#     cerebro.adddata(data)
#     # 只传入过滤后的策略参数
#     cerebro.addstrategy(CONFIG['strategy']['class'], **strategy_params)

#     initial_capital = CONFIG['initial_capital']
#     cerebro.broker.setcash(initial_capital)
#     cerebro.broker.setcommission(commission=CONFIG['commission'])

#     # 添加常用分析器，包括 PyFolio 用于后续量化指标计算
#     # cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
#     # cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
#     cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
#     cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
#     cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

#     results = cerebro.run()
#     strat = results[0]
#     final_value = cerebro.broker.getvalue()
#     profit = final_value - initial_capital
#     roi = (profit / initial_capital) * 100

#     # 获取 PyFolio 的回测收益率数据
#     portfolio_stats = strat.analyzers.pyfolio.get_pf_items()
#     returns = portfolio_stats[0]
    
#     # 修改这里：确保索引没有时区信息，避免使用tz_convert
#     # 首先检查是否有时区信息再进行处理
#     try:
#         if hasattr(returns.index, 'tz') and returns.index.tz is not None:
#             returns.index = returns.index.tz_localize(None)
#     except:
#         # 如果无法处理时区，创建一个新的无时区索引
#         try:
#             returns = pd.Series(returns.values, index=pd.DatetimeIndex(returns.index.astype('datetime64[ns]')))
#         except:
#             # 如果依然失败，使用更简单的方法
#             returns = pd.Series(returns.values, index=pd.DatetimeIndex([str(idx) for idx in returns.index]))

#     # 计算量化指标（完整的收益指标）
#     qs_stats = {}
#     try:
#         qs_stats["Sharpe Ratio"] = qs.stats.sharpe(returns)
#         qs_stats["Sortino Ratio"] = qs.stats.sortino(returns)
#         qs_stats["Calmar Ratio"] = qs.stats.calmar(returns)
#         qs_stats["Max Drawdown"] = qs.stats.max_drawdown(returns)
#         qs_stats["Win Rate"] = qs.stats.win_rate(returns)
#         qs_stats["Profit Factor"] = qs.stats.profit_factor(returns)
#         qs_stats["Expected Return (M)"] = qs.stats.expected_return(returns, aggregate='M')
#         qs_stats["Kelly Criterion"] = qs.stats.kelly_criterion(returns)
#         qs_stats["Risk of Ruin"] = qs.stats.risk_of_ruin(returns)
#         qs_stats["Tail Ratio"] = qs.stats.tail_ratio(returns)
#         qs_stats["Common Sense Ratio"] = qs.stats.common_sense_ratio(returns)
#         qs_stats["Average Win"] = qs.stats.avg_win(returns)
#         qs_stats["Average Loss"] = qs.stats.avg_loss(returns)
#         qs_stats["Annualized Volatility"] = qs.stats.volatility(returns, periods=252)
#         qs_stats["Skew"] = qs.stats.skew(returns)
#         qs_stats["Kurtosis"] = qs.stats.kurtosis(returns)
#         qs_stats["Value at Risk"] = qs.stats.value_at_risk(returns)
#         qs_stats["Conditional VaR"] = qs.stats.conditional_value_at_risk(returns)
#         qs_stats["Payoff Ratio"] = qs.stats.payoff_ratio(returns)
#         qs_stats["Gain to Pain Ratio"] = qs.stats.gain_to_pain_ratio(returns)
#         qs_stats["Ulcer Index"] = qs.stats.ulcer_index(returns)
#         qs_stats["Consecutive Wins"] = qs.stats.consecutive_wins(returns)
#         qs_stats["Consecutive Losses"] = qs.stats.consecutive_losses(returns)
#         # ----------------- 新增指标 -----------------
#         qs_stats["Avg Return"] = qs.stats.avg_return(returns)
#         qs_stats["CAGR"] = qs.stats.cagr(returns)
#         qs_stats["Expected Shortfall"] = qs.stats.expected_shortfall(returns)
#         qs_stats["Information Ratio"] = qs.stats.information_ratio(returns)
#         qs_stats["Profit Ratio"] = qs.stats.profit_ratio(returns)
#         qs_stats["R2"] = qs.stats.r2(returns)
#         qs_stats["R Squared"] = qs.stats.r_squared(returns)
#         qs_stats["Recovery Factor"] = qs.stats.recovery_factor(returns)
#         qs_stats["Risk-Return Ratio"] = qs.stats.risk_return_ratio(returns)
#         qs_stats["Win/Loss Ratio"] = qs.stats.win_loss_ratio(returns)
#         qs_stats["Worst"] = qs.stats.worst(returns)
#         # ------------------------------------------------
#     except Exception as e:
#         qs_stats["error"] = str(e)

#     # 整合基础回测指标与量化收益指标
#     backtest_results = {
#         "Initial Capital": initial_capital,
#         "Final Value": final_value,
#         "Profit": profit,
#         "ROI (%)": roi,
#     }
#     backtest_results.update(qs_stats)

#     return backtest_results

In [11]:
# 新增清理函数：清理不在最终结果 CSV 中的 optuna 数据库文件
# def clean_incomplete_optuna_db_files(config):
#     """
#     清理不在最终优化结果 CSV 中的 optuna 数据库文件。
#     该函数会读取最终结果 CSV（如果存在），提取已完成优化的 (Symbol, Target Timeframe) 组合，
#     然后删除 reports 目录下不在该列表中的 optuna 数据库文件。
#     """
#     import os
#     from glob import glob

#     # 从最终结果 CSV 中加载已完成优化组合
#     master_file, master_df, optimized_combinations = load_master_results(config)
#     print(f"已完成优化组合: {optimized_combinations}")
    
#     # 匹配与当前策略相关的 optuna 数据库文件
#     pattern = os.path.join(config['reports_path'], f"optuna_{config['strategy']['name']}_*.db")
#     db_files = glob(pattern)
#     for db_file in db_files:
#         filename = os.path.basename(db_file)
#         prefix = f"optuna_{config['strategy']['name']}_"
#         # 确保文件名格式正确
#         if filename.startswith(prefix) and filename.endswith(".db"):
#             # 去掉前缀和后缀，得到 "symbol_timeframe"
#             core = filename[len(prefix):-3]  # 去掉后面的 ".db"
#             parts = core.rsplit("_", 1)
#             if len(parts) != 2:
#                 continue
#             symbol, timeframe = parts
#             # 如果此组合不在最终结果中，则删除该数据库文件
#             if (symbol, timeframe) not in optimized_combinations:
#                 try:
#                     os.remove(db_file)
#                     print(f"已删除未完成优化的数据库文件：{db_file}")
#                 except Exception as e:
#                     print(f"删除文件 {db_file} 出错：{e}")
#     print("数据库清理完成。")

In [12]:
# def restore_from_backup(master_file, backup_folder='backups'):
#     """
#     从备份恢复优化结果文件
#     """
#     # 检查备份文件夹
#     if not os.path.exists(backup_folder):
#         print(f"备份文件夹不存在: {backup_folder}")
#         return False
    
#     # 查找与主文件相关的所有备份
#     filename = os.path.basename(master_file)
#     backup_pattern = os.path.join(backup_folder, f"{filename}.*.bak")
#     import glob
#     backup_files = glob.glob(backup_pattern)
    
#     if not backup_files:
#         print(f"没有找到备份文件: {backup_pattern}")
#         return False
    
#     # 按修改时间排序，获取最新的备份
#     backup_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
#     latest_backup = backup_files[0]
    
#     try:
#         # 读取备份文件
#         if latest_backup.endswith('.csv.bak'):
#             backup_df = pd.read_csv(latest_backup)
#         else:
#             backup_df = pd.read_excel(latest_backup)
        
#         # 读取当前文件(如果存在)
#         current_df = pd.DataFrame()
#         if os.path.exists(master_file):
#             try:
#                 current_df = pd.read_csv(master_file)
#             except:
#                 print(f"无法读取当前文件: {master_file}")
        
#         # 比较记录数
#         if current_df.empty or len(backup_df) > len(current_df):
#             print(f"备份文件包含 {len(backup_df)} 条记录，当前文件包含 {len(current_df)} 条记录")
            
#             # 创建当前文件的备份（以防万一）
#             if os.path.exists(master_file) and not current_df.empty:
#                 timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#                 current_backup = f"{master_file}.before_restore.{timestamp}"
#                 import shutil
#                 shutil.copy2(master_file, current_backup)
#                 print(f"在恢复前已创建当前文件的备份: {current_backup}")
            
#             # 恢复备份文件
#             import shutil
#             shutil.copy2(latest_backup, master_file)
#             print(f"已从备份 {latest_backup} 恢复数据")
#             return True
#         else:
#             print(f"当前文件记录数({len(current_df)})不少于备份文件({len(backup_df)})，无需恢复")
#             return False
#     except Exception as e:
#         print(f"恢复过程中出错: {str(e)}")
#         return False


In [ ]:
if __name__ == '__main__':
    # 先清理不完整的 Optuna 数据库文件（不在最终结果 CSV 中的组合）
    # clean_incomplete_optuna_db_files(CONFIG)
    batch_optimize(CONFIG)

已自动创建备份: backups\optimization_results_MeanReverterShort_20240101-20250208.csv.20250309_043119.bak
剩余需要优化的组合数量: 91 个
开始处理组合: SOLUSDT-15min
开始处理组合: XRPUSDT-15min
开始处理组合: DOGEUSDT-15min
开始处理组合: BNBUSDT-15min
开始处理组合: OMUSDT-15min
开始处理组合: BCHUSDT-15min
开始处理组合: ENAUSDT-15min
开始处理组合: TRUMPUSDT-15min
开始处理组合: SUIUSDT-15min
开始处理组合: 1000PEPEUSDT-15min
开始处理组合: AVAXUSDT-15min
开始处理组合: LINKUSDT-15min
开始处理组合: ACTUSDT-15min
开始处理组合: AAVEUSDT-15min
开始处理组合: TRXUSDT-15min
开始处理组合: ONDOUSDT-15min
开始处理组合: BNXUSDT-15min
开始处理组合: WLDUSDT-15min
开始处理组合: PNUTUSDT-15min
开始处理组合: UNIUSDT-15min
开始处理组合: FILUSDT-15min
开始处理组合: JUPUSDT-15min
开始处理组合: KAITOUSDT-15min
开始处理组合: TAOUSDT-15min
开始处理组合: IPUSDT-15min
开始处理组合: IPUSDT-15min
开始处理组合: LAYERUSDT-15min
开始处理组合: 1000CHEEMSUSDT-15min
开始处理组合: TSTUSDT-15min
开始处理组合: USUALUSDT-15min


[I 2025-03-09 04:31:33,822] A new study created in memory with name: SOLUSDT_15min


开始优化 SOLUSDT-15min...


[I 2025-03-09 04:38:53,804] Trial 14 finished with value: 7.483183153286902 and parameters: {'frequency': 12, 'rsiFrequency': 44, 'sellZoneDistance': 7, 'avgUpATRSum': 7, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 54, 'pyramiding': 9}. Best is trial 14 with value: 7.483183153286902.


[SOLUSDT-15min] Trial 14: 参数 {'frequency': 12, 'rsiFrequency': 44, 'sellZoneDistance': 7, 'avgUpATRSum': 7, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 54, 'pyramiding': 9} -> 得分 7.48


...

[I 2025-03-09 14:25:36,845] Trial 126 finished with value: 15.162271578843495 and parameters: {'frequency': 8, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 6, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 46, 'pyramiding': 6}. Best is trial 25 with value: 21.84076345430083.
[I 2025-03-09 14:25:37,738] Trial 128 finished with value: 14.038965836372789 and parameters: {'frequency': 10, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 6, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 46, 'pyramiding': 6}. Best is trial 25 with value: 21.84076345430083.


[AXSUSDT-15min] Trial 126: 参数 {'frequency': 8, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 6, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 46, 'pyramiding': 6} -> 得分 15.16
[AXSUSDT-15min] Trial 128: 参数 {'frequency': 10, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 6, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 46, 'pyramiding': 6} -> 得分 14.04


[I 2025-03-09 14:25:38,777] Trial 124 finished with value: 15.162271578843495 and parameters: {'frequency': 8, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 6, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 46, 'pyramiding': 6}. Best is trial 25 with value: 21.84076345430083.


[AXSUSDT-15min] Trial 124: 参数 {'frequency': 8, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 6, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 46, 'pyramiding': 6} -> 得分 15.16
[AXSUSDT-15min] Trial 130: 参数 {'frequency': 10, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 5, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 48, 'pyramiding': 6} -> 得分 15.94


[I 2025-03-09 14:26:00,303] Trial 130 finished with value: 15.935943354284596 and parameters: {'frequency': 10, 'rsiFrequency': 38, 'sellZoneDistance': 5, 'avgUpATRSum': 5, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 48, 'pyramiding': 6}. Best is trial 25 with value: 21.84076345430083.


In [42]:
# # 数据恢复工具 - 如果发现结果文件数据被意外清空，运行此单元格恢复数据

# # 构造结果文件路径
# start_clean = CONFIG['start_date'].replace("-", "")
# end_clean = CONFIG['end_date'].replace("-", "")
# results_file = os.path.join(
#     CONFIG['reports_path'], 
#     CONFIG['results_filename_template'].format(
#         strategy_name=CONFIG['strategy']['name'],
#         start_date=start_clean,
#         end_date=end_clean
#     )
# )

# # 检查当前文件状态
# current_records = 0
# if os.path.exists(results_file):
#     try:
#         current_df = pd.read_csv(results_file)
#         current_records = len(current_df)
#     except:
#         print(f"无法读取当前结果文件: {results_file}")
# print(f"当前结果文件状态: {'存在' if os.path.exists(results_file) else '不存在'}, 包含 {current_records} 条记录")

# # 从最新备份恢复
# if current_records == 0 or input("是否要从备份恢复数据？(y/n): ").lower() == 'y':
#     if restore_from_backup(results_file):
#         print("数据恢复成功！")
#     else:
#         print("数据恢复失败。尝试从另一个备份文件夹恢复...")
#         if restore_from_backup(results_file, backup_folder='backups_final'):
#             print("数据从final备份成功恢复！")
#         else:
#             print("所有恢复尝试均失败，请手动处理。")